In [1]:
# Используемые библиотеки
import yaml
import csv
from collections import defaultdict
import os

In [ ]:
# Считывание нахождения в директории
file_path_yaml = os.getenv("file_path_yaml")
file_path_csv = os.getenv("file_path_csv")

In [3]:
def load_yaml(file_path):
    '''Функция для чтения .yaml'''
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)

In [4]:
def load_logs(file_path):
    '''Функция для чтения .csv'''
    logs = []
    with open(file_path, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            logs.append(row['node_id'])
    return logs

In [ ]:
def find_statistics_nodes(folders):
    '''Функция для поиска по узлу'''
    #Мноежство для хранения результатов
    statistics_nodes = set()
    #Функция для проверки текущей ссылки на link
    def traverse(node):
        if 'link' in node:
            return
        if node.get('name') == 'statistics':
            statistics_nodes.add(node)
            return
        #Рекурсивный обход дочерних узлов
        for key, child in node.get('values', {}).items():
            traverse(child)
    #Рекурсивный обход узлов
    for key, node in folders.items():
        traverse(node)
    return statistics_nodes

In [ ]:
def get_all_node_ids(node):
    '''Функция для получения id узла'''
    node_ids = set()
    #Функция для проверки отношения узла с link
    def collect_ids(node):
        if 'link' in node:
            return
        if 'values' in node:
            #Рекурсивный обход дочерних ключей
            for key, child in node['values'].items():
                node_ids.add(key)
                collect_ids(child)
    collect_ids(node)
    return node_ids

In [7]:
folders = load_yaml(file_path_yaml)
logs = load_logs(file_path_csv)
#Поиск узла 'statistics' в структуре
statistics_nodes = set()
def find_nodes(node, path):
    #Поиск элементов statistics
    if 'name' in node and node['name'] == 'statistics':
        statistics_nodes.add(node)
    #Поиск ссылок на другие пути
    if 'values' in node:
        for key, child in node['values'].items():
            find_nodes(child, path + [key])
    #Рекурсия поиска statistics
    for key, node in folders.items():
        find_nodes(node, [key])
    if not statistics_nodes:
        print("Узел 'statistics' не найден в структуре.")
        return
    #Соберем все идентификаторы узлов, принадлежащих 'statistics'
    target_ids = set()
    for node in statistics_nodes:
        target_ids.update(get_all_node_ids(node))
        #Добавляем также сам узел 'statistics', если у него есть идентификатор
        #В данном случае, у нас нет прямого доступа к идентификатору узла, поэтому нужно иначе
        #Пересмотрим подход

    #Альтернативный подход: соберем все идентификаторы узлов, которые ведут в 'statistics'
    #Для этого нужно знать идентификатор узла 'statistics'
    #В sample.yaml 'statistics' находится в двух местах: 
    #- /home/statistics (идентификатор 9cae29)
    #- /tmp/statistics (идентификатор 82cc0d)
home_statistics_id = '9cae29'
tmp_statistics_id = '82cc0d'
#В задаче требуется папка /home/statistics, поэтому берем 9cae29
target_ids = set()
def collect_home_statistics_ids(folders):
    ids = set()
    def traverse(node, current_id):
        if 'link' in node:
            return
        if 'values' in node:
            for key, child in node['values'].items():
                if current_id == home_statistics_id:
                    ids.add(key)
                    traverse(child, key)
        for key, node in folders.items():
            traverse(node, key)
        return ids
#Добавляем сам узел statistics 
target_ids = collect_home_statistics_ids(folders)
target_ids.add(home_statistics_id)
#Подсчет обращений
count = 0
for node_id in logs:
    if node_id in target_ids:
        count += 1
print(count)

TypeError: expected str, bytes or os.PathLike object, not NoneType